In [1]:
%matplotlib inline

In [2]:
import torch
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
import PIL
from torch.nn.utils import weight_norm
from torch.autograd import Variable

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from os.path import join
import scipy.special
import pandas as pd
import seaborn as sn
import scipy
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from tqdm.notebook import tqdm
import random
from csv import DictWriter
import nengolib
# if gpu is to be used
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor

IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
ttype = FloatTensor

import seaborn as sns
print(use_cuda)
import pickle


True


In [3]:
sn.set_context("poster")

In [4]:

class coRNNCell(nn.Module):
    def __init__(self, n_inp, n_hid, dt, gamma, epsilon):
        super(coRNNCell, self).__init__()
        self.dt = dt
        self.gamma = gamma
        self.epsilon = epsilon
        self.i2h = nn.Linear(n_inp + n_hid + n_hid, n_hid)

    def forward(self,x,hy,hz):
        hz = hz + self.dt * (torch.tanh(self.i2h(torch.cat((x, hz, hy),1)))
                                   - self.gamma * hy - self.epsilon * hz)
        hy = hy + self.dt * hz

        return hy, hz

class coRNN(nn.Module):
    def __init__(self, n_inp, n_hid, n_out, dt, gamma, epsilon):
        super(coRNN, self).__init__()
        self.n_hid = n_hid
        self.cell = coRNNCell(n_inp,n_hid,dt,gamma,epsilon)
        self.readout = nn.Linear(n_hid, n_out)

    def forward(self, x):
        outputs = []
        ## initialize hidden states
        hy = Variable(torch.zeros(x.size(1),self.n_hid)).to(device)
        hz = Variable(torch.zeros(x.size(1),self.n_hid)).to(device)

        for t in range(x.size(0)):
            hy, hz = self.cell(x[t],hy,hz)
            outputs.append(hy)
        outputs = torch.stack(outputs)
        
        output = self.readout(outputs)

        return output

# Load Stimuli

In [6]:
import collections

def mackey_glass(sample_len=1000, tau=17, delta_t=10, seed=None, n_samples=1):
    # Adapted from https://github.com/mila-iqia/summerschool2015/blob/master/rnn_tutorial/synthetic.py
    '''
    mackey_glass(sample_len=1000, tau=17, seed = None, n_samples = 1) -> input
    Generate the Mackey Glass time-series. Parameters are:
        - sample_len: length of the time-series in timesteps. Default is 1000.
        - tau: delay of the MG - system. Commonly used values are tau=17 (mild 
          chaos) and tau=30 (moderate chaos). Default is 17.
        - seed: to seed the random generator, can be used to generate the same
          timeseries at each invocation.
        - n_samples : number of samples to generate
    '''
    history_len = tau * delta_t 
    # Initial conditions for the history of the system
    timeseries = 1.2
    
    if seed is not None:
        np.random.seed(seed)

    samples = []

    for _ in range(n_samples):
        history = collections.deque(1.2 * np.ones(history_len) + 0.2 * \
                                    (np.random.rand(history_len) - 0.5))
        # Preallocate the array for the time-series
        inp = np.zeros((sample_len,1))
        
        for timestep in range(sample_len):
            for _ in range(delta_t):
                xtau = history.popleft()
                history.append(timeseries)
                timeseries = history[-1] + (0.2 * xtau / (1.0 + xtau ** 10) - \
                             0.1 * history[-1]) / delta_t
            inp[timestep] = timeseries
        
        # Squash timeseries through tanh
        inp = np.tanh(inp - 1)
        samples.append(inp)
    return samples


def generate_data(n_batches, length, split=0.5, seed=0,
                  predict_length=15, tau=17, washout=100, delta_t=1,
                  center=True):
    X = np.asarray(mackey_glass(
        sample_len=length+predict_length+washout, tau=tau,
        seed=seed, n_samples=n_batches))
    X = X[:, washout:, :]
    cutoff = int(split*n_batches)
    if center:
        X -= np.mean(X)  # global mean over all batches, approx -0.066
    Y = X[:, predict_length:, :]
    X = X[:, :-predict_length, :]
    assert X.shape == Y.shape
    return ((X[:cutoff], Y[:cutoff]),
            (X[cutoff:], Y[cutoff:]))

In [7]:
(train_X, train_Y), (test_X, test_Y) = generate_data(128, 5000)
dataset = torch.utils.data.TensorDataset(torch.Tensor(train_X).cuda(), torch.Tensor(train_Y).cuda())
dataset = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

dataset_valid = torch.utils.data.TensorDataset(torch.Tensor(test_X).cuda(), torch.Tensor(test_Y).cuda())
dataset_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=64, shuffle=False)

print(train_X.shape, train_Y.shape, test_X.shape)

(64, 5000, 1) (64, 5000, 1) (64, 5000, 1)


## Setup for Model

In [8]:

def train(model, ttype, train_loader, test_loader, optimizer, loss_func, epoch, perf_file,
          loss_buffer_size=800, batch_size=4, device='cuda',
          prog_bar=None):
    
    assert(loss_buffer_size%batch_size==0)
        
    losses = []
    last_test_perf = 0
    best_test_perf = 1000
    
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        data = data.to(device).transpose(1,0)
        target = target.to(device)
        optimizer.zero_grad()
        out = model(data).transpose(1,0)
        loss = loss_func(out,
                         target)

        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().numpy())
        losses = losses[int(-loss_buffer_size/batch_size):]
        
        if ((batch_idx*batch_size)%loss_buffer_size == 0):
            loss_track = {}
            last_test_perf = test_model(model, 'cuda', test_loader, 
                                  )
            loss_track['avg_loss'] = np.mean(losses)
            loss_track['last_test'] = last_test_perf
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            with open(perf_file, 'a+') as fp:
                csv_writer = DictWriter(fp, fieldnames=list(loss_track.keys()))
                if fp.tell() == 0:
                    csv_writer.writeheader()
                csv_writer.writerow(loss_track)
                fp.flush()
            if best_test_perf > last_test_perf:
                torch.save(model.state_dict(), perf_file[:-4]+".pt")
                best_test_perf = last_test_perf
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.4f},valid: {:.4f}"
            format_list = [e,batch_idx*batch_size, np.mean(losses), 
                           last_test_perf]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
                
def test_model(model, device, test_loader):
    # Test the Model
    nrmsd = []
    with torch.no_grad():
        for x, y in test_loader:
            data = x.to(device).transpose(1,0)
            target = y.to(device)
            optimizer.zero_grad()
            out = model(data).transpose(1,0)
            nrmsd.append(nengolib.signal.nrmse(out.detach().cpu().numpy().flatten(), 
                                               target=target.detach().cpu().numpy().flatten()))

    perf = np.array(nrmsd).mean()
    return perf

In [5]:
cornn_params = dict(n_inp=2,
                    n_hid=128, 
                    n_out=1,
                    dt=1.6e-2,
                    gamma=94.5,
                    epsilon=9.5)
model = coRNN(**cornn_params).cuda()

tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)


Total Weights: 33281
coRNN(
  (cell): coRNNCell(
    (i2h): Linear(in_features=258, out_features=128, bias=True)
  )
  (readout): Linear(in_features=128, out_features=1, bias=True)
)


In [24]:
inp, target = next(enumerate(dataset_valid))[1]
print(model(inp.transpose(1,0)).shape)
print(target.shape)

torch.Size([5000, 64, 1])
torch.Size([64, 5000, 1])


In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-2)
loss_func = nn.MSELoss()

epochs = 1000
batch_size = 32
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
last_perf = 1000
for e in progress_bar:
    train(model, ttype, dataset, dataset_valid, 
          optimizer, loss_func, batch_size=batch_size, loss_buffer_size=64,
          epoch=e, perf_file=join('perf','mackeyglass_coRNN_9.csv'),
          prog_bar=progress_bar)

KeyboardInterrupt: 

In [13]:
debug

> <ipython-input-12-b159598f1528>(9)<module>()
      5 for e in progress_bar:
      6     last_perf = train(model, ttype, train_X, train_Y, test_X, test_Y, 
      7                       optimizer, loss_func, batch_size=batch_size, last_perf=last_perf,
      8                       epoch=e, perf_file=join('perf','mackeyglass_LSTM_3.csv'),
----> 9                       prog_bar=progress_bar, loss_buffer_size=32)

--KeyboardInterrupt--
ipdb> quit


In [ ]:
dat = pd.read_csv(join('perf', 'mackeyglass_deepsith_.csv'))
maxpres = 64
dat['presnum_epoch'] = ((dat.batch_idx) + maxpres*dat.epoch)/maxpres


fig = plt.figure(figsize=(20,7))
ax = fig.add_subplot(1,2,1)
sn.lineplot(data=dat, x=dat.presnum_epoch, y='last_testperf', 
            color='blue', ax=ax)

ax.grid(True)
ax.set_title(dat.last_testperf.min())
ax.legend(["Training", 
           ])
ax.set_xlim(0, 500)
ax.set_ylim(.00, .05)
ax = fig.add_subplot(1,2,2)
sn.lineplot(data=dat, x=dat.presnum_epoch, y='avg_loss', 
            color='blue',ax=ax)

ax.set_xlim(0, 500)
ax.set_ylim(.00, .0001)
ax.legend(["Test NRMSD"])
ax.grid(True)


In [ ]:
import pickle

In [ ]:
with open('LMU_data.dil', 'rb') as f:
    dats = pickle.load(f)

In [ ]:
fig = plt.figure(figsize=(12,7))

plt.plot(np.arange(0, 500, 1), dats[0], linewidth=3)
plt.plot(np.arange(0, 500, 1), dats[1], linewidth=3)
plt.plot(np.arange(0, 500, 1), dats[2], linewidth=3)
plt.plot(np.arange(0, 500, 1), perf['loss'][::32], color='purple', linewidth=3)
plt.yscale('log')
plt.legend(['LSTM', 'LMU', 'Hybrid', 'DeepSITH'])
plt.savefig('performance')
sns.despine(offset=15)
plt.savefig('mackey_glass_perf', dpi=200)

In [ ]:
# Impulse-based SITH class
class iSITH():
    def __init__(self, tau_min=.1, tau_max=20., buff_max=100, k=8, ntau=10, dt=.01, g=0.0):
        # save inputs
        self.k = k
        self.tau_min = tau_min
        self.tau_max = tau_max
        self.ntau = ntau
        self.dt = dt
        self.g = g        # calc tau_star
        self.c = (self.tau_max/self.tau_min)**(1./(ntau-1))-1
        self.tau_star = self.tau_min * (1 + self.c)**np.arange(ntau)        # determine the time points in steps of dt
        self.times = np.arange(dt, buff_max+dt, dt)        # fill the ftilde filters
        a = np.log(k)*k
        b = np.log(np.arange(2,k)).sum()
        A = ((1/self.tau_star)*(np.exp(a-b))*(self.tau_star**self.g))[:, np.newaxis]
        self.filters = A*np.exp((np.log(self.times[np.newaxis, :]/self.tau_star[:, np.newaxis])*(k+1)) + \
                                (k*(-self.times[np.newaxis, :]/self.tau_star[:, np.newaxis])))        #A = ((1/self.tau_star)*(k**(k)/factorial(k-1))*(self.tau_star**self.g))[:, np.newaxis]
        #self.filters = A*((self.times[np.newaxis, :]/self.tau_star[:, np.newaxis])**(k+1)) * \
        #                np.exp(k*(-self.times[np.newaxis, :]/self.tau_star[:, np.newaxis]))    def present(self, x):
        # make sure input is features by time
        x = np.atleast_2d(x)        # get num points to save from the convolution
        xlen = x.shape[-1]        # loop over features and taustars performing the convolution
        # note we're scaling the output by both dt and the k/(k+1)
        return np.array([[signal.convolve(self.filters[i], x[j])[:xlen]
                            for i in range(len(self.filters))]
                         for j in range(len(x))])*self.dt*self.k/(self.k+1)

In [ ]:
model.eval()
fig = plt.figure(figsize=(15,4))

idx = 9
out = model(test_X[idx].view(1,1,1,-1))[0]
plt.plot(test_Y[idx][-1000:].detach().cpu().numpy(), linewidth=5)
plt.plot(out[-1000:].detach().cpu().numpy())
plt.legend(['Test Out', 'DS Prediction'])
model.train();
print('Normalized RMSD:', np.sqrt(((test_Y[idx].detach().cpu().numpy() - out.detach().cpu().numpy())**2).mean()/
                                  (test_Y[idx].detach().cpu().numpy()**2).mean()))
sns.despine(offset=15)

plt.savefig('long_boy')

In [ ]:
model.eval()
out = model(train_X[idx].view(1,1,1,-1))[0]
plt.plot(train_Y[idx][-1000:].detach().cpu().numpy())
plt.plot(out[-1000:].detach().cpu().numpy())
model.train();
print('Normalized RMSD:', np.sqrt(((train_Y[idx].detach().cpu().numpy() - out.detach().cpu().numpy())**2).mean()/
                                  (train_Y[idx].detach().cpu().numpy()**2).mean()))

In [ ]:
model.eval()
nrmsd = []
for idx in range(test_X.shape[0]):
    out = model(test_X[idx].view(1,1,1,-1))[0]
    nrmsd.append(np.sqrt(((test_Y[idx].detach().cpu().numpy() - out.detach().cpu().numpy())**2).mean()/
                                  (test_Y[idx].detach().cpu().numpy()**2).mean()))
model.train();
i
print('Avg Normalized RMSD:', np.array(nrmsd).mean())

In [ ]:
model.eval()
def_scale = 5000
res = {'nrmsd':[],
       'sig_leng':[]}
for num in range(1000, 10000, 500):
    nrmsd = []
    #_, (test_X, test_Y) = generate_data(128, num)
    #train_X = ttype(train_X).view(64, 1, 1, num)
    #train_Y = ttype(train_Y).view(64, num, 1)
    #test_Xhalf = ttype(test_X).view(64, 1, 1, num)
    #test_Yhalf = ttype(test_Y).view(64, num, 1)
    for idx in range(test_X.shape[0]):
        f = scipy.interpolate.interp1d(np.arange(0, def_scale, 1),
                                       test_X[idx].view(-1).detach().cpu())
        new_X = f(np.arange(0, def_scale-1*(num/def_scale), 1/(num/def_scale)))
        
        f = scipy.interpolate.interp1d(np.arange(0, def_scale, 1),
                                       test_Y[idx].view(-1).detach().cpu())
        new_Y = f(np.arange(0, def_scale-1*(num/def_scale), 1/(num/def_scale)))
        
        out = model(ttype(new_X).view(1,1,1,-1))[0]
        nrmsd.append(np.sqrt(((new_Y - out.view(-1).detach().cpu().numpy())**2).mean()/
                                      (new_Y**2).mean()))
    res['nrmsd'].append(np.array(nrmsd).mean())
    res['sig_leng'].append(num)
    print('Avg Normalized RMSD at {}:'.format(num), np.array(nrmsd).mean())
model.train();
dat = pd.DataFrame(res)
dat.to_pickle("DS_RES_MG.dill")

In [ ]:
dat_2 = pd.read_pickle(open("LOLA_RES_MG.dill", 'rb'))
dat = pd.read_pickle(open("DS_RES_MG.dill", 'rb'))


In [ ]:
fig = plt.figure(figsize=(10,7))

plt.plot(np.arange(1000, 9600, 500)/5000, dat['nrmsd'])
plt.plot(np.arange(1000, 9600, 500)/5000, dat_2['nrmsd'])
plt.legend(['DeepSITH', 'LoLa'])
plt.xlabel("scale")
plt.ylabel("NRMSD")
plt.savefig("LoLavsDS")